In [1]:
### Fixing import errors of the

import sys
import os

# This code navigates up one directory from the notebook's location ('examples/')
# to get the project's root directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# We check if the path is already in the system path.
# If not, we add it to the beginning of the list.
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added project root to Python path: {project_root}")
else:
    print(f"Project root is already in Python path: {project_root}")

# Optional: You can print the first few paths to verify
print("\nVerifying sys.path:")
for i, path in enumerate(sys.path[:5]):
    print(f"{i}: {path}")

Added project root to Python path: /home/nick/projects/Llama-Index-GliREL-GraphRAG

Verifying sys.path:
0: /home/nick/projects/Llama-Index-GliREL-GraphRAG
1: /usr/lib/python312.zip
2: /usr/lib/python3.12
3: /usr/lib/python3.12/lib-dynload
4: 


In [2]:
import json
from llama_index.core import Document
from src.GlirelPathExtractor import GlirelPathExtractor 
from src.RecursivePathExtractor import RecursiveLLMPathExtractor
from llama_index.core import SimpleDirectoryReader, PropertyGraphIndex,Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
import nest_asyncio
import openlit


In [3]:
llm = Ollama(
    model= "gemma3:12b",
    request_timeout=120.0,
    context_window=8128,
    temperature=0.0
)

Settings.llm = llm
Settings.chunk_size=512
Settings.chunk_overlap=64

embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed2:latest",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.embed_model = embed_model

In [4]:
with open('../.data/novel.json', 'r') as file:
    # Load the JSON data from the file into a Python object
    data = json.load(file)

In [5]:
print(data[0]["corpus_name"])

Novel-30752


In [6]:
documents = [Document(text=t["context"],id_=t["corpus_name"]) for t in data]

In [7]:
doc = documents[1]
doc.id_

'Novel-51410'

In [8]:
extractorGli = GlirelPathExtractor(device="cuda")
extractorLLM = RecursiveLLMPathExtractor(llm=Settings.llm,
    num_workers=1,
    max_paths_per_chunk=15)


In [9]:
# for starting later kill 1 263 nodes
#documents = documents[4:]
documents[0].id_

'Novel-30752'

In [10]:
nest_asyncio.apply()
for mode in ["gli", "hybrid","llm"]:
    if mode == "gli":
        kg_ext = [extractorGli]
    if mode == "hybrid":
        kg_ext = [extractorGli,extractorLLM]
    if mode == "llm":
        kg_ext = [extractorLLM]
    
    openlit.init(
        otlp_endpoint="http://127.0.0.1:4318",
        application_name=f"{mode}3",
        environment="obama_enviroment"    
    )
    
    for document in documents:
        index = PropertyGraphIndex.from_documents(
            documents=[document],
            kg_extractors=kg_ext,
            use_async = False,   
        )
        novel_id = document.id_
        index.storage_context.persist(persist_dir=f"./.persistent_storage/.storage_context/{mode}/{novel_id}")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

/home/nick/envs/grag-llama/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
2025-07-23 02:07:33.129 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...
2025-07-23 02:07:33.274 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...
2025-07-23 02:07:33.375 | WARNING  | glirel.spacy_integration:__cal

NO ENTITIES FOUND FOR THIS NODE!!!


2025-07-23 02:07:33.814 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...


NO ENTITIES FOUND FOR THIS NODE!!!


...

NO ENTITIES FOUND FOR THIS NODE!!!


2025-07-23 06:52:20.159 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...
2025-07-23 06:52:21.807 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...


NO ENTITIES FOUND FOR THIS NODE!!!
Number of nodes: 119
Is extraction complete? False
Starting extraction loop #1
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Starting extraction loop #1
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Starting extraction loop #1
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Starting extraction loop #1
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Starting extraction loop #1
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Starting extraction loop #1
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
St

OpenLIT metrics setup failed. Metrics will not be available: 'NoneType' object has no attribute 'create_histogram'


Number of nodes: 88
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Starting extraction loop #1
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Max number of relations per chunk found.
node has been analyzed
Is extraction complete? False
Max number of relations per chunk found.
node 